[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/main/vizro-ai/examples/dashboard_by_vizro_ai.ipynb)

In [1]:
!pip install openai==1.55.3 httpx==0.27.2 AzureOpenAI --force-reinstall --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/

#### Setup the LLM API key

# Use Vizro-AI to generate Vizro dashboard
# **At the top of this notebook, click `Runtime` -> `Run all`**

In [2]:
import os

from google.colab import userdata

# @title ## Setup
# @markdown #### 1. Install Vizro-AI
print("Installing dependencies")
%pip install uv -q -q -q
# %pip uninstall ipykernel -y -q -q -q
# %pip install -U ipykernel -q -q -q
!uv pip install --system vizro-ai --quiet
print("✅ Installation finished")

# @markdown #### 2. Setup LLM API access


def _safe_get_userdata(key):
    try:
        return userdata.get(key)
    except Exception as e:
        print(f"Warning: Unable to access {key}. Reason: {e!s}")
        return None


# Always try to set the OPENAI_API_KEY
api_key = _safe_get_userdata("OPENAI_API_KEY")
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("❌ OPENAI_API_KEY not set. Click `Secrets` icon on the left to setup.")

# Conditionally set OPENAI_BASE_URL if provided and accessible
openai_base_url = _safe_get_userdata("OPENAI_BASE_URL")
if openai_base_url:
    os.environ["OPENAI_BASE_URL"] = openai_base_url
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

api_version = _safe_get_userdata("API_VERSION")
if openai_base_url:
    os.environ["API_VERSION"] = api_version
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

print("\nCurrent environment variables:")
print(f"OPENAI_API_KEY: {'✅ Set' if 'OPENAI_API_KEY' in os.environ else 'Not set'}")
print(f"OPENAI_BASE_URL: {'✅ Set' if 'OPENAI_BASE_URL' in os.environ else 'Not set by user, use default'}")

Installing dependencies
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 56.9 MB/s eta 0:00:00
✅ Installation finished

Current environment variables:
OPENAI_API_KEY: ✅ Set
OPENAI_BASE_URL: ✅ Set


In [3]:
# @markdown #### Data upload (multiple dataframes accepted)
# @markdown - click `Choose Files` if you'd like to upload and use your own data
# @markdown - or click `Cancel upload` and use the default example data (plotly gapminder) for initial exploration

import pandas as pd
from google.colab import files
from plotly.express.data import gapminder

uploaded = files.upload()

dfs = []

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
    dfs.append(df)
else:
    for fn in uploaded.keys():
        print(f'User uploaded file "{fn}"')

        df_uploaded = pd.read_csv(fn)
        dfs.append(df_uploaded)

Saving sample_csv_2.csv to sample_csv_2.csv
User uploaded file "sample_csv_2.csv"


In [6]:
from IPython.display import HTML, display
from ipywidgets import Layout, Textarea

# @title ## Input the LLM model choice and user prompt { run: "auto" }

# @markdown #### 1. Choose the LLM model

LLM = "gpt-4o-mini"  # @param ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini"]

# @markdown ---
# @markdown #### 2. Input your dashboard question
# @markdown e.g. "Create a gdp dashboard with 1 page. use a bar chart to show gdp per continent and filter data by country"

default_user_input = """  plot me 2 dashboard one with pencil sales and other sith rnd predictions
"""
"""
# Create a 2 page dashboard.

# <page 1> 1 bar chart, 1 filter
# use a bar chart to show the average GDP per continent. Add a filter to filter the bar chart by country.

# <page 2> 1 card, 1 table
# create a card with text "This is the Gapminder dataset"
# use a table to show the data details, showing features like GDP and life expectancy.

# page 2 layout: Image the page consists 6 grid units horizontally.
# The card takes 1 unit of the page space on the left and the table takes 5 units on the right.
# """
user_input = Textarea(
    value=default_user_input,
    placeholder="Input your requirements for the dashboard",
    description="user_input:",
    disabled=False,
    rows=10,  # This sets the number of visible lines
    layout=Layout(width="auto", height="auto"),  # This makes the width automatic
)
display(user_input)
display(HTML("<br>"))

# @markdown ---


print(f"Selected LLM: {LLM}")
# print(f"User input: {user_input.value}")

Textarea(value='  plot me 2 dashboard one with pencil sales and other sith rnd predictions\n', description='us…

Selected LLM: gpt-4o-mini


In [9]:
import os
from vizro_ai import VizroAI
from langchain_openai import AzureChatOpenAI

client = AzureChatOpenAI(
        api_key= os.environ["OPENAI_BASE_URL"],
        azure_endpoint=os.environ["OPENAI_BASE_URL"],
        api_version=os.environ["API_VERSION"],
    )
vizro_ai = VizroAI(model=client)


In [10]:
res = vizro_ai.plot(dfs[0], user_input.value, return_elements=True)

In [11]:
res = vizro_ai.dashboard(dfs, user_input.value, return_elements=True)

Store df info:   0%|          | 0/1 [00:00<?, ?it/s]

df_name: company_profits


Generate dashboard plan:   0%|          | 0/2 [00:00<?, ?it/s]

Building page: Pencil Sales:   0%|          | 0/5 [00:00<?, ?it/s]

Building page: RD Predictions:   0%|          | 0/5 [00:00<?, ?it/s]

Currently Building ... [Page] <Pencil Sales> components:   0%|          | 0/1 [00:00<?, ?it/s]

Currently Building ... [Page] <RD Predictions> components:   0%|          | 0/1 [00:00<?, ?it/s]

Currently Building ... [Page] <Pencil Sales> controls: 0it [00:00, ?it/s]

Currently Building ... [Page] <RD Predictions> controls: 0it [00:00, ?it/s]

In [13]:
# @title ## Build the dashboard using Vizro-AI

import ipywidgets as widgets
from IPython.display import display


def _run_code(button):
    print("Running VizroAI...")
    import dash._callback_context
    from vizro import Vizro
    from vizro_ai import VizroAI

    dash._callback_context.context_value.set({})
    Vizro._reset()

    vizro_ai = VizroAI(model=client)
    res = vizro_ai.dashboard(dfs, user_input.value, return_elements=True)
    print(res)

    vizro_dashboard = Vizro().build(res.dashboard)

    button1.result = vizro_dashboard
    print("✅ Dashboard generated. Check its code and live view below")
    print(res.code)


button1 = widgets.Button(
    description="Build Dashboard!",
    button_style="info",
    tooltip="Click to build the dashboard using Vizro-AI",
)
display(button1)

button1.on_click(_run_code)

Button(button_style='info', description='Build Dashboard!', style=ButtonStyle(), tooltip='Click to build the d…

Running VizroAI...


Store df info:   0%|          | 0/1 [00:00<?, ?it/s]

df_name: business_profits


Generate dashboard plan:   0%|          | 0/2 [00:00<?, ?it/s]

...

Currently Building ... [Page] <RD Predictions> components:   0%|          | 0/1 [00:00<?, ?it/s]

Currently Building ... [Page] <Pencil Sales> components:   0%|          | 0/1 [00:00<?, ?it/s]

Currently Building ... [Page] <Pencil Sales> controls: 0it [00:00, ?it/s]

Currently Building ... [Page] <RD Predictions> controls: 0it [00:00, ?it/s]

DashboardOutputs(dashboard=Dashboard(id='12e0c8b2-bad6-40fb-1948-8dec4f65d4d9', pages=[Page(id='Pencil Sales', components=[Graph(id='rnd_spend_vs_profit', type='graph', figure=vizro_ai.plot._response_models.rnd_spend_vs_profit(data_frame='business_profits'), title='', header='', footer='', actions=[])], title='Pencil Sales', description='', layout=Layout(id='b4862b21-fb97-d435-8856-1712e8e5216a', grid=[[0]], row_gap='24px', col_gap='24px', row_min_height='0px', col_min_width='0px'), controls=[], path='/pencil-sales', actions=[]), Page(id='RD Predictions', components=[Graph(id='rnd_predictions', type='graph', figure=vizro_ai.plot._response_models.rnd_predictions(data_frame='business_profits'), title='', header='', footer='', actions=[])], title='RD Predictions', description='', layout=Layout(id='259f4329-e6f4-590b-9a16-4106cf6a659e', grid=[[0]], row_gap='24px', col_gap='24px', row_min_height='0px', col_min_width='0px'), controls=[], path='/rd-predictions', actions=[])], theme='vizro_dar

In [14]:
# @title ## Render dashboard

import ipywidgets as widgets
from IPython.display import display


def _run_dashboard(button):
    button1.result.run()


button2 = widgets.Button(
    description="Run Dashboard!",
    button_style="info",
    tooltip="Click to render the Vizro dashboard",
)
display(button2)

button2.on_click(_run_dashboard)

Button(button_style='info', description='Run Dashboard!', style=ButtonStyle(), tooltip='Click to render the Vi…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>